# Comparación red neuronal regresión

## Importar librerías

In [16]:
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_breast_cancer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
!pip install -q keras-tuner
import keras_tuner as kt
import numpy as np
import tensorflow as tf
import random as random

## Importar datos

In [17]:
california_housing = fetch_california_housing(as_frame=True)

df = california_housing.frame

## Análisis explotario de datos

In [18]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [19]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [20]:
# Ver distribución de las variables
import matplotlib.pyplot as plt

# df.hist(bins=30, figsize=(15, 10))
# plt.tight_layout()
# plt.show()

In [21]:
# Escalar los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop(columns=['MedHouseVal']))
y = df['MedHouseVal']

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Función para crear el modelo
def build_model(hp):
    model = Sequential()
    
    num_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    
    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=16, max_value=128, step=16)
        activation = hp.Choice(f'activation_{i}', values=['relu', 'tanh', 'sigmoid'])
        if i == 0:
            model.add(Dense(units=units, activation=activation, input_shape=(X_train.shape[1],)))
        else:
            model.add(Dense(units=units, activation=activation))
            
    model.add(Dense(1))  # Capa de salida para regresión
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [24]:
turner = kt.BayesianOptimization(
    build_model,
    objective='val_mae',
    max_trials=5,
    num_initial_points=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='california_housing_regression',
    seed=42
)

turner.search(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

best_hps = turner.get_best_hyperparameters(num_trials=1)[0]
print(f"Mejores hiperparámetros: {best_hps.values}")

Trial 5 Complete [00h 00m 13s]
val_mae: 0.34917381405830383

Best val_mae So Far: 0.34917381405830383
Total elapsed time: 00h 00m 56s
Mejores hiperparámetros: {'num_layers': 5, 'units_0': 64, 'activation_0': 'tanh', 'units_1': 112, 'activation_1': 'sigmoid', 'units_2': 32, 'activation_2': 'relu', 'units_3': 80, 'activation_3': 'relu', 'units_4': 64, 'activation_4': 'sigmoid'}


In [25]:
# Modelo final con los mejores hiperparámetros
model = turner.hypermodel.build(best_hps)

cp = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[cp, es], verbose=0)

test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Precisión en el conjunto de prueba: {test_accuracy[1]}")

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Precisión en el conjunto de prueba: 0.34064340591430664
